## **Machine Learning**<br>
<b><i>Regression</b></i><br><br>
Housing Prices

In [5]:
# pip install category_encoders

In [6]:
import pandas as pd
X_full = pd.read_csv('https://raw.githubusercontent.com/redsonlopez/machine_learning/main/housing_prices_train.csv', index_col='Id')
X_test_full = pd.read_csv('https://raw.githubusercontent.com/redsonlopez/machine_learning/main/housing_prices_test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice', "LotFrontage"], axis=1, inplace=True)

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)

from category_encoders import TargetEncoder
target_encoder = TargetEncoder()
target_encoder.fit(X_train.select_dtypes('object'), y_train)

# Transform the categorical columns in training and test sets
train_encoded = target_encoder.transform(X_train.select_dtypes('object'))
test_encoded = target_encoder.transform(X_valid.select_dtypes('object'))

# Combine the numeric columns with the encoded categorical columns in training and validation sets
X_train_encoded = pd.concat([X_train.select_dtypes(exclude='object'), train_encoded], axis=1)
X_valid_encoded = pd.concat([X_valid.select_dtypes(exclude='object'), test_encoded], axis=1)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train_encoded.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

MasVnrArea      6
GarageYrBlt    58
dtype: int64


In [7]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=1)
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_encoded))
X_valid_imputed = pd.DataFrame(imputer.transform(X_valid_encoded))

from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=1)
model.fit(X_train_imputed, y_train)
predictions = model.predict(X_valid_imputed)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(predictions, y_valid)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 128275.54641347389
